In [2]:
import numpy as np
import pandas as pd
import json

# W207 Final Project
# Data Processing

## 7 August 2018

### Introduction  
The data for this project is sourced through various users using the OpenBarbell device and collected by gym owner Jordan Berke. As granted by the owner, no NDA is required and we are free to work with the data as we please. The data is stored as json and the schema is openly shared by OpenBarbell through their [github wiki page](https://github.com/squatsandsciencelabs/OpenBarbell-V3/wiki/OpenBarbell-Data-Storage-Format).

The raw data contains unclean data, data from outdated devices, and nested data structures that need to be unpacked. Due to the complexity of the data processing required, this data pipeline is held within its own notebook. At the end of the pipeline, a light-weight and clean csv file will be produces to serve as the base for all future operations.

# 1. Import the Raw Data  
This can take 5-10 minutes as the raw json file is quite large. This import also showcases the need for a simpler, cleaner data file to work off of. It is simply not productive to wait this long for whenever we need to iterate through the workflow.

In [3]:
df_raw = pd.read_json('../obdatadump.json', lines = True)

# 2. Data Processing and Cleaning  
The major issues to address within our pipeline:  
- **Invalid Data Rows**
    - There are various flags within the data that explicitly indicate when a row is invalid. Additionally, we have established logical thresholds for particular data fields so that we can reliably weed out rows with extremely erroneous data 
- **Cleaning User Input**
    - Various fields are provided through user input. As these fields are optional, freeform, and unstandardized, there can be a lot of issues with the data. It is especially troubling since these fields represent our labels and important features.
- **Unnesting Inner Data Structures**
    - The raw data is hierarchical, with repetition data for a set being stored as a list in each set row. Furthermore, within the repetition data is another list for various data fields. All of this information needs to be unpacked to reveal all the various datapoints available in the data.
- **Joining the Data Together**
    - Following up on unnesting the data, we then need to join all of that data back together. The goal is to have a single row per repetition, containing columns for its parent set, the repetition itself, and repetition detail.
- **Re-Index the Data**
    - Unique identifiers are already provided for sets and repetitions. However, they are overly complex and are tightly coupled with the raw data. Simple integer IDs refactored to the transformed data are more useful for our project.


## 2.1 Processing Set Data  
Please refer to the inline comments for more information about this step

In [6]:
raw_set_ct = df_raw.shape[0]
print(f"{raw_set_ct} rows in the raw set data.")

# Remove rows that are invalid. This includes rows flagged as
# removed, flagged as deleted, or have null exercise values
print("Pruning invalid rows...")
df_set = df_raw.loc[df_raw['removed'] == 0]
df_set = df_set.loc[(df_set['deleted'].isnull()) | (df_set['deleted'] == 0)]
df_set = df_set.loc[df_set['exercise'].notnull()]

# Create clean exercise labels for our data. A label is determined as 'clean'
# if it matches one of the qualifying strings. This helps remove variations
# of the big 3 lifts. We then remove any rows that we are not able to cleanly
# label
clean_bench = ['bench', 'bench press', 'bp', 'competition bench', 'comp bench']
clean_squat = ['squat', 'back squat', 'competition squat', 'comp squat']
clean_deadlift = ["deadlift", "sumo deadlift", "conventional deadlift", "comp deadlift", "competition deadlift"]
def clean_exercise(e):
    e = e.lower().strip()
    if e in clean_bench:
        return 'bench'
    if e in clean_squat:
        return 'squat'
    if e in clean_deadlift:
        return 'deadlift'
    return 'other'
    
df_set['exercise_clean'] = df_set['exercise'].apply(clean_exercise)
df_set = df_set.loc[df_set['exercise_clean'] != 'other']

# Coerce datetime fields into their correct type
print("Coercing datetime fields..")
date_cols = ['endTime', 'startTime','initialStartTime']
df_set[date_cols] = df_set[date_cols].apply(pd.to_datetime, errors = 'coerce')

# Rate of perceived exertion (rpe) is entered through user input. This 
# requires some extra effort on our end to clean the data. This includes 
# delimitting, parsing, and capping values.
print("Fixing rpe values...")
def fix_rpe(r):
    rvals = []
    if type(r) == str:
        for rval in r.split('-'):
            try:
                rvals.append(np.minimum(float(rval.replace(',', '.')), 10))
            except:
                continue
    
    if type(r) in (float, int):
        rvals.append(np.minimum(r, 10))
    
    return np.mean(rvals) if len(rvals) > 0 else np.nan

df_set['rpe_num'] = df_set['rpe'].apply(fix_rpe)
                             
# Weight is also entered through user input and needs some
# special processing. We account for european notation, delimited
# lists, and cap outliers at 1000 lbs.
print("Fixing weight values...")
def fix_weight(row, metric_col):
    wvals = []
    conversion_factor = 1 if row[metric_col] == 'lbs' else 2.20462
    w = row['weight']
    
    if type(w) == str:
        w = w.replace(',','.') if w.count(',') == 1 else w
        dlm = '-' if w.count('-') > 0 else ','
        dlm = '.' if w.count('.') > 1 else dlm
        
        for wval in w.split(dlm):
            if dlm == '.' and (wval == '' or wval == '5'):
                continue
                             
            try:
                wvals.append(np.minimum(float(wval) * conversion_factor, 1000))
            except:
                continue
    
    if type(w) in ('int', 'float'):
        wvals.append(np.minimum(w * conversion_factor, 1000))
    
    return np.mean(wvals) if len(wvals) > 0 else np.nan

df_set['weight_lbs'] = df_set.apply(fix_weight, axis=1, metric_col='metric')
df_set = df_set.loc[df_set['weight_lbs'] > 0]

print(f"Set data processing complete! {df_set.shape[0]} rows after processing.")

127832 rows in the raw set data.
Pruning invalid rows...
Coercing datetime fields..
Fixing rpe values...
Fixing weight values...
Set data processing complete! 53115 rows after processing.


## 2.2 Processing Rep Data  
Please refer to the inline comments for more information about this step.

In [8]:
# Pull setID and reps data into its own dataframe
df_rep = df_set[['setID','reps']]

# Reps are nested within a dict for each set row. For our purposes, we
# want to explode out each rep into an individual row. To do that we
# apply a dict parser to pull out the reps and concatenate them back
# together into a dataframe
print("Parsing out reps...")
dfs = []
def dict_to_df(row, dict_col):
    dict_df = pd.DataFrame.from_dict(row['reps'])
    dfs.append(dict_df.assign(**row.drop(dict_col)))

df_rep.apply(dict_to_df, axis=1, dict_col='reps')
print("Aggregating rep data...")
df_rep = pd.concat(dfs)
raw_rep_ct = df_rep.shape[0]
print(f"{raw_rep_ct} rows in the raw rep data.")

# There is are additional nested data fields within the rep data that
# needs to be parsed out into their own columns. After parsing and
# appending these data fields, drop the original raw rep data
print("Parsing rep data fields...")
colnames = ['StartMessg', 'RepN', 'AvgVel', 'ROM', 'PeakVel', 'PeakVelLoc', 'PeakAccel', 'RepDur'
            , 'TimeBWReps', 'TimeRepComp', 'TimeRepWait', 'SlowAllow', 'Backlight','MinAllow']
df_rep[colnames] = pd.DataFrame([row[:14] for row in df_rep['data'].values.tolist()], index=df_rep.index)
df_rep.drop(columns=['data'], inplace=True)

# Remove invalid rows. This includes reps not flagged as 'isValid', flagged
# as 'removed'.
print("Pruning invalid rows..") #219626
df_rep = df_rep.loc[(df_rep['isValid'] == True) & (df_rep['removed'] == False)]

# Through meta-analysis of the file, we have determined that >86% of our data
# uses OpenBarbellv3. We felt this was sufficient to ensure good modeling. 
# Since there are numerous differences between each major version, standardizing 
# our data to just a single version also helps make life easier.
df_rep = df_rep.loc[df_rep['appVersion'].apply(lambda x: str(x)[0] == '3')]

# Coerce the numeric fields into their correct type
print("Coercing numeric values..") #219626
num_cols = ['RepN', 'AvgVel', 'ROM', 'PeakVel', 'PeakVelLoc', 'PeakAccel', 'RepDur', 'TimeBWReps'
            , 'TimeRepComp', 'TimeRepWait', 'SlowAllow', 'MinAllow']
df_rep[num_cols] = df_rep[num_cols].apply(pd.to_numeric, errors = 'coerce')
    
# Some of the velocity fields have 'infinity' values. Replace them with nans
df_rep = df_rep.replace(np.inf, np.nan)

# After correcting the numeric fields, we can perform additional validity checks.
# Primarily checking if the numeric fields are within an acceptable range.
df_rep = df_rep.loc[(df_rep['AvgVel'] > 0) & (df_rep['AvgVel'] <= 3)]
df_rep = df_rep.loc[df_rep['ROM'] <= 2000]
df_rep = df_rep.loc[(df_rep['PeakAccel'] <= 3000) | (df_rep['PeakVel'] <= 10)]
df_rep = df_rep.loc[(df_rep['PeakVelLoc'] > 0) & (df_rep['PeakVelLoc'] <= 100)]

print(f"Rep processing complete! {df_rep.shape[0]} rows after processing.")

Parsing out reps...
Aggregating rep data...
211277 rows in the raw rep data.
Parsing rep data fields...
Pruning invalid rows..
Coercing numeric values..
Rep processing complete! 181224 rows after processing.


### 2.3 Combining Set and Rep Data  
At this step, both our set and rep dataframes have completed processing and cleaning. We can now merge the two by joining on the 'setID'. We apply an inner join, requiring a match between both dataframes. Any columns that appear in both dataframes and given a suffix to help identify its original source.

After joining, we essentially have all of our data finalized. It is at this point we can re-index our rows to make future processing easier. We factorize the setID across the entire dataframe and factorize RepN within each set grouping. To follow standard indexing practice, we start the indices at 1.

Next, we check to see just how much of the original data was pruned from the beginning of our pipeline to the end.

In [12]:
print("Combining set and rep data...")
df_full = df_set.merge(df_rep, on ="setID", suffixes=('_set', '_rep'))

df_full['setID_ix'] = df_full['setID'].factorize()[0] + 1
df_full['RepCount'] = df_full.groupby('setID_ix')['RepN'].transform(lambda x: pd.factorize(x)[0] + 1)

print(f"Full data processing complete! {df_full.shape[0]} rows after processing.")
dropped = 1 - (df_full.shape[0]/ raw_rep_ct)
print("Percent of data rows dropped after cleaning: {:.2%}".format(dropped))

Combining set and rep data...
Full data processing complete! 181224 rows after processing.
Percent of data rows dropped after cleaning: 14.22%


# 3 Export to CSV  
Our final step is to simply write out our final dataframe to a csv file in our working directory. This csv file will be referenced for the remaining sections of our project.

In [13]:
print("Exporting to final csv...")
df_full.to_csv('./ob_data_w207_filtered.csv')

Exporting to final csv...
